# Access AXDS API and download Smartfin data

In [2]:
import json
import requests
import urllib
import os
import pandas as pd

### Data are on stage.platforms.axds.co

In [3]:
PLATFORM_API = 'https://stage.platforms.axds.co/'
params = { 'verbosity': 2 }

### Get session identifiers from CSV and find matching sessions in Axiom database
Use example session ID to develop process

In [16]:
session_ids = pd.read_csv('session_id_list.csv', header = None)
session_ids

,0,1
0,google_101440910315486450418_000666D321BE_1803...,NaN
1,google_101440910315486450418_000666D321BE_1803...,NaN
2,google_105992916073328834624_000666D321BB_1711...,NaN
3,google_113399247482189200727_000666D32296_1711...,NaN
4,google_113399247482189200727_000666D32296_1711...,NaN
...,...,...
136,google_102847823526503707395_000666D6B7FD_1802...,NaN
137,google_102847823526503707395_000666DD9894_1910...,NaN
138,google_102847823526503707395_000666DD9894_1910...,NaN
139,google_114729917292219509612_000666DD7DE4_1809...,NaN


### Loop through each session and grab motion/ocean CSVs
Shove CSVs into subdirectories named after their session IDs

In [18]:
# Largest loop, through each session
for session in session_ids[0]:
    search_tag = f'packrat_source_id:{session}*'
    sessions = requests.get(f'{PLATFORM_API}/tags/search/{search_tag}', params=params).json()['tags']
    
#     print(session)
    # For each session, find associated CSVs
    csv_files = []
    for k, c in sessions.items():
        files = list(c.values())[0]['files']
        for fname, fc in files.items():
            if fname.endswith('csv'):
                csv_files.append(
                    fc['url']
                )
#     display(f'Found {len(csv_files)} CSV files for this session')

    # url is of form https://files.stage.platforms.axds.co/smartfin/legacy/google_112421847946536791050_0006667E22B1_170621130002/motion.csv
    for url in csv_files:
        
        url_split = url.split('/') 
        new_fname = url_split[-2]+'_'+url_split[-1] # session ID + ocean or motion CSV

        # make a new subdirectory for files from this session
        new_dir = os.path.join('data', url_split[-2])
        if not os.path.exists(new_dir): os.mkdir(new_dir)

        path = os.path.join('data', url_split[-2], new_fname)
        urllib.request.urlretrieve(url, path)